In [3]:
import boto3

import pandas as pd
import numpy as np
import math
from sagemaker import get_execution_role

role = get_execution_role()

s3 = boto3.client("s3")
resource = boto3.resource("s3")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [88]:
gdp_food_household_merge = pd.read_csv("new_gdp_food_household_merge_no_county3.csv")
gdp_food_household_merge

,County,hotel_locations,towns_in_county,Percent_trees,Percent_shrub,Percent_urban,rivers_in_county,roads_in_county,Area (km2),Population (2019 Census),...,grid_battery_pct,grid_solar_pct,avg_electricity_use_grid,avg_typical_hours_elect_grid,avg_worst_hours_elect_grid,not_grid_generator_pct,not_grid_battery_pct,not_grid_solar_pct,avg_distance_to_connect,GDP percapita in US dollar (PPP)
0,nairobi,1.0,1.0,0.268004,0.252397,0.243705,7.0,8.0,694.9,4397073.0,...,0.011513,0.097859,60.912899,7.521720,8.242618,0.000000,0.000000,0.000000,128.050729,6321.932795
1,nakuru,8.0,4.0,0.038993,0.056162,0.003039,36.0,24.0,7509.5,2162202.0,...,0.000000,0.000000,37.304289,4.715640,7.190349,0.000000,0.000000,0.000000,41.263336,4778.924448
2,kiambu,1.0,1.0,0.120034,0.115989,0.018661,23.0,12.0,2449.2,2417735.0,...,0.006359,0.092395,76.571541,4.510192,4.778392,0.018372,0.027343,0.375615,116.374842,3484.666434
3,mombasa,0.0,1.0,0.112710,0.026204,0.113001,4.0,3.0,212.5,1208333.0,...,0.000000,0.000000,62.928659,3.990792,4.278126,0.000000,0.000000,0.000000,127.550000,5488.553238
4,nyandarua,1.0,0.0,0.121343,0.038516,0.000755,19.0,4.0,3107.7,638289.0,...,0.000000,0.000000,56.325000,11.500000,12.000000,0.000000,0.210623,0.570865,72.980000,7670.506620
5,machakos,0.0,1.0,0.013195,0.057392,0.003830,37.0,18.0,5952.9,1421932.0,...,0.000000,0.056194,53.061738,8.730830,8.317618,0.042179,0.004213,0.481725,17.603868,3270.198575
6,meru,6.0,1.0,0.106703,0.099744,0.001043,67.0,11.0,7003.1,1545714.0,...,0.000000,0.029793,43.715090,10.336842,11.592562,0.008979,0.213840,0.418793,10.608683,2966.266722
7,kisumu,2.0,1.0,0.001078,0.003286,0.026750,21.0,17.0,2009.5,1155574.0,...,0.000000,0.163266,69.736693,11.673978,16.342454,0.020105,0.047468,0.465816,79.408140,3360.234827
8,bungoma,1.0,1.0,0.120128,0.071628,0.001635,32.0,10.0,2206.9,1670570.0,...,0.000000,0.192905,41.483431,13.806565,13.865024,0.000000,0.059532,0.318408,14.242899,2193.263377
9,kakamega,1.0,1.0,0.057303,0.000000,0.001035,40.0,13.0,3033.8,1867579.0,...,0.009178,0.265065,50.243125,13.312374,14.517568,0.000000,0.084398,0.328245,92.813103,1951.724666


# Analyse the Nan Values

In [94]:
# Count NaN values in each column
nan_count = gdp_food_household_merge.isna().sum()

# Display the results
print(nan_count)

County                              0
hotel_locations                     0
towns_in_county                     0
Percent_trees                       0
Percent_shrub                       0
Percent_urban                       0
rivers_in_county                    0
roads_in_county                     0
Area (km2)                          0
Population (2019 Census)            0
avg_household_size                  0
avg_household_per_dwelling          0
pct_own_dwelling                    0
avg_rooms_occupied                  0
percent_grid_connect                0
percent_any_elect                   0
percent_bank_account                0
percent_mobile_money                0
total_percent_batteries             0
total_percent_solar                 0
total_percent_generator             0
grid_generator_pct                  0
grid_battery_pct                    0
grid_solar_pct                      0
avg_electricity_use_grid            0
avg_typical_hours_elect_grid        0
avg_worst_ho

# Remove the Kenya row

In [74]:
gdp_food_household_merge = gdp_food_household_merge.drop(47)

In [91]:
gdp_food_household_merge.columns

Index(['County', 'hotel_locations', 'towns_in_county', 'Percent_trees',
       'Percent_shrub', 'Percent_urban', 'rivers_in_county', 'roads_in_county',
       'Area (km2)', 'Population (2019 Census)', 'avg_household_size',
       'avg_household_per_dwelling', 'pct_own_dwelling', 'avg_rooms_occupied',
       'percent_grid_connect', 'percent_any_elect', 'percent_bank_account',
       'percent_mobile_money', 'total_percent_batteries',
       'total_percent_solar', 'total_percent_generator', 'grid_generator_pct',
       'grid_battery_pct', 'grid_solar_pct', 'avg_electricity_use_grid',
       'avg_typical_hours_elect_grid', 'avg_worst_hours_elect_grid',
       'not_grid_generator_pct', 'not_grid_battery_pct', 'not_grid_solar_pct',
       'avg_distance_to_connect', 'GDP percapita in US dollar (PPP)'],
      dtype='object')

# Visualizing the GDP (ppp) vs other valriables

In [95]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# List of independent variables (x_vars)
x_vars = ['County', 'Area (km2)', 'Population (2019 Census)', 'avg_household_size',
          'percent_grid_connect', 'avg_distance_to_connect']

# Create a single visualization with various plot types
fig, axes = plt.subplots(3, 2, figsize=(20, 20))
fig.suptitle('Relationship between Independent Variables and GDP (PPP)',fontsize=20)

# Define the space between the title and plots (adjust as needed)
top_space = 0.78  # You can increase this value to add more space

for i, var in enumerate(x_vars):
    row, col = i // 2, i % 2
    ax = axes[row, col]

    if gdp_food_household_merge[var].dtype == 'object':
        # For categorical variables like 'County,' create a bar chart of the actual values
        categories = gdp_food_household_merge[var].unique()
        values = [gdp_food_household_merge[gdp_food_household_merge[var] == category]['GDP in US dollar millions (PPP)'].values for category in categories]

        ax.bar(categories, [value[0] for value in values])
        ax.set_xlabel(var)
        ax.set_ylabel('GDP (PPP)')
        ax.set_xticklabels(categories, rotation=45, ha="right", fontsize=12)  # Adjust rotation, alignment, and font size
    else:
        # For numeric variables, use a scatter plot
        sns.scatterplot(x=gdp_food_household_merge[var], y=gdp_food_household_merge['GDP in US dollar millions (PPP)'], ax=ax)
        ax.set_xlabel(var)
        ax.set_ylabel('GDP (PPP)')
        

plt.tight_layout()
plt.show()
fig.savefig("EDA.png")

KeyError: 'GDP in US dollar millions (PPP)'

# Visualizing GDP VS Energy Access

In [92]:
import seaborn as sns

# List of numeric independent variables (x_vars)
x_vars = ['Area (km2)', 'Population (2019 Census)', 'avg_household_size',
          'percent_grid_connect', 'avg_distance_to_connect']

# Combine the target variable and the selected independent variables
plot_data = gdp_food_household_merge[x_vars + ['GDP in US dollar millions (PPP)']]

# Create a pair plot
sns.pairplot(plot_data)
plt.show()

KeyError: "['GDP in US dollar millions (PPP)'] not in index"

In [80]:
import sagemaker
from sagemaker.s3 import S3Uploader

# Specify your local directory and S3 bucket name
local_directory = "HDE_UN_Datathon_23"
s3_bucket = "hartreedataexplorers"

# Upload the local directory to S3
S3Uploader.upload(local_directory+"/model_outputs", f"s3://{s3_bucket}/model_outputs")

print("Files uploaded to S3 successfully.")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


FileNotFoundError: [Errno 2] No such file or directory: 'HDE_UN_Datathon_23/model_outputs'